In [1]:
!dir
#!rm -r MaskTrack_RCNN
#!git clone https://github.com/umd-fire-coml/MaskTrack_RCNN

import os
from os.path import join, isfile
import re
import skimage.io
import numpy as np
import sys
import random
import math
import re
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from time import time

%matplotlib qt5 

# Root directory of the project
ROOT_DIR = os.path.abspath("./")
# Root directory of the dataset
DATA_DIR = os.path.abspath("../dataset/wad")
sys.path.append(ROOT_DIR)  # To find local version of the library
# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")

# Import Mask RCNN
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from mrcnn import config, utils



# Local path to trained coco weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

 Volume in drive C is Windows
 Volume Serial Number is C67A-6D67

 Directory of C:\Users\tmthy\Documents\prog\python3\coml\MaskTrack_RCNN\project

06/18/18  23:32    <DIR>          .
06/18/18  23:32    <DIR>          ..
06/18/18  22:49    <DIR>          .ipynb_checkpoints
06/18/18  22:33             2,760 inspect_wad_data.ipynb
06/18/18  23:27    <DIR>          logs
06/18/18  22:48       257,557,808 mask_rcnn_coco.h5
06/18/18  23:23            11,576 wad_data.py
06/18/18  23:32            90,021 wad_data_training.ipynb
06/18/18  23:23    <DIR>          __pycache__
               4 File(s)    257,662,165 bytes
               5 Dir(s)  119,161,159,680 bytes free


'export' is not recognized as an internal or external command,
operable program or batch file.
c:\program files\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
!dir MaskTrack_RCNN

 Volume in drive C is Windows
 Volume Serial Number is C67A-6D67

 Directory of C:\Users\tmthy\Documents\prog\python3\coml\MaskTrack_RCNN\project



File Not Found


## Configuration

In [3]:
from project import wad_data
config = wad_data.WADConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                48
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9

## Dataset

load_mask has been tested. green light .. green light

In [4]:
# Training dataset
dataset_train = wad_data.WADDataset()

dataset_train.load_data(DATA_DIR, "train")

dataset_train.prepare()

print(dataset_train.image_info)

dataset_val = wad_data.WADDataset()
 
dataset_val.load_data(DATA_DIR, "val")

dataset_val.prepare()


[{'id': '170908_061502408_Camera_5', 'source': 'WAD', 'path': '170908_061502408_Camera_5.jpg', 'mask_path': '170908_061502408_Camera_5_instanceIds.png'}, {'id': '170908_061502408_Camera_6', 'source': 'WAD', 'path': '170908_061502408_Camera_6.jpg', 'mask_path': '170908_061502408_Camera_6_instanceIds.png'}, {'id': '170908_061502547_Camera_5', 'source': 'WAD', 'path': '170908_061502547_Camera_5.jpg', 'mask_path': '170908_061502547_Camera_5_instanceIds.png'}, {'id': '170908_061502547_Camera_6', 'source': 'WAD', 'path': '170908_061502547_Camera_6.jpg', 'mask_path': '170908_061502547_Camera_6_instanceIds.png'}, {'id': '170908_061502686_Camera_5', 'source': 'WAD', 'path': '170908_061502686_Camera_5.jpg', 'mask_path': '170908_061502686_Camera_5_instanceIds.png'}, {'id': '170908_061502825_Camera_5', 'source': 'WAD', 'path': '170908_061502825_Camera_5.jpg', 'mask_path': '170908_061502825_Camera_5_instanceIds.png'}, {'id': '170908_061502825_Camera_6', 'source': 'WAD', 'path': '170908_061502825_Ca

In [5]:
# print("Image Count (train): {}".format(len(dataset_train.image_ids)))
# print("Class Count: {}".format(dataset_train.num_classes))

In [6]:
# !df -h
# !cat /proc/cpuinfo
#!cat /proc/meminfo
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir=(LOGS_DIR+'{}').format(time()))

In [7]:
# import numpy as np
# image_ids = np.random.choice(dataset_val.image_ids, 4)

# #for image_id in image_ids:
# image_id = image_ids[0]
# image = dataset_val.load_image(image_id)
# mask, class_ids = dataset_val.load_mask(image_id)
# visualize.display_top_masks(image, mask, class_ids, dataset_val.class_names)

In [8]:
# # Compute Bounding box
# bbox = utils.extract_bboxes(mask)

# # Display image and additional stats
# print("image_id ", image_id, dataset_train.image_reference(image_id))
# log("image", image)
# log("mask", mask)
# log("class_ids", class_ids)
# log("bbox", bbox)
# # Display image and instances
# visualize.display_instances(image, bbox, mask, class_ids, dataset_train.class_names)

## Training

In [9]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=LOGS_DIR)

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: C:\Users\tmthy\Documents\prog\python3\coml\MaskTrack_RCNN\project\logs\wad20180618T2332\mask_rcnn_wad_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_co

c:\program files\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1


C:\Users\tmthy\AppData\Roaming\Python\Python36\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
C:\Users\tmthy\AppData\Roaming\Python\Python36\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
C:\Users\tmthy\AppData\Roaming\Python\Python36\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


ResourceExhaustedError: OOM when allocating tensor with shape[2,512,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: rpn_model_1/rpn_conv_shared/convolution = Conv2D[T=DT_FLOAT, _class=["loc:@train...propFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](fpn_p3/BiasAdd, rpn_conv_shared/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: proposal_targets/roi_assertion_1/AssertGuard/Assert/Switch_1/_1721 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_5242_proposal_targets/roi_assertion_1/AssertGuard/Assert/Switch_1", tensor_type=DT_FLOAT, _de

In [ ]:
#model.train(dataset_train, dataset_val, 
#            learning_rate=config.LEARNING_RATE / 10,
#            epochs=1, 
#            layers="all")